In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.DCVC_net import DCVC_net
import torch
from torchvision import transforms
import numpy as np
import pathlib
import os
import matplotlib.pyplot as plt
import wandb
import tqdm

/home/jatin/miniconda3/envs/DCVC/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LAMBDA = 2048
BATCH_SIZE = 4
DATA_DIR = pathlib.Path('/data2/jatin/vimeo_septuplet/sequences')
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

In [4]:
video_net = DCVC_net(up_strategy='default')

480


In [5]:
# load the good weights
video_net.opticFlow = torch.load('../DCVC-Old/optflow.pth')

video_net = video_net.to(DEVICE)

optimizer = torch.optim.AdamW(video_net.parameters(), lr=1e-4)


In [6]:
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, crop_size=256, make_b_cut=True, deterministic=False):
        self.data_dir = data_dir
        self.crop_size = crop_size
        self.make_b_cut = make_b_cut
        self.deterministic = deterministic
        self.all_paths = []
        for seq in os.listdir(self.data_dir):
            subseq = os.listdir(self.data_dir / seq)
            for s in subseq:
                self.all_paths.append(self.data_dir / seq / s)
        assert len(self.all_paths) == 91701
        
        self.transforms = torch.nn.Sequential(
            transforms.RandomCrop(crop_size)
        )
       
    def __getitem__(self, i):
        path = self.all_paths[i]
        imgs = []
        if self.make_b_cut:
            # load two reference frames and the B-frame in the middle
            #TODO: implement making this deterministic
            interval = np.random.randint(1, 4) # can be 1, 2, or 3
            ref1 = plt.imread(path / f'im{1}.png')
            ref2 = plt.imread(path / f'im{1 + interval*2}.png')
            # this is the B-frame, in the middle
            im = plt.imread(path / f'im{1 + interval}.png')
            imgs = [ref1, ref2, im]
        else:
            # load full sequence
            for i in range(1, 8):
                # should be between [0, 1]
                img = plt.imread(path / f'im{i}.png')
        
        # plt.imread should make inputs in [0, 1] for us
        imgs = np.stack(imgs, axis=0)
        # bring RGB channels in front
        imgs = imgs.transpose(0, 3, 1, 2)
        return self.transforms(torch.FloatTensor(imgs))

    def __len__(self):
        return len(self.all_paths)

ds = VideoDataset(DATA_DIR)

In [7]:
mse_criterion = torch.nn.MSELoss()

In [8]:
x = ds[0]
x = x.unsqueeze(dim=0)
x = x.to(DEVICE)

In [9]:
ref1 = x[:,0]
ref2 = x[:,1]
im = x[:,2]

In [12]:
# TEST MEMC
preds_dict = video_net(ref1, ref2, im, compress_type='train_compress', train_type='memc')
mse1 = mse_criterion(preds_dict['pred1'], im)
mse2 = mse_criterion(preds_dict['pred2'], im)
mse = (mse1 + mse2)/2
mse.backward()
optimizer.step()

In [13]:
# TEST MEMC + MV BPP
preds_dict = video_net(ref1, ref2, im, compress_type='train_compress', train_type='memc_bpp')
mse1 = mse_criterion(preds_dict['pred1'], im)
mse2 = mse_criterion(preds_dict['pred2'], im)
mse = (mse1 + mse2)/2
loss = mse * LAMBDA + preds_dict['mv_z_bpp'] + preds_dict['mv_y_bpp']
loss.backward()
optimizer.step()

In [16]:
# TEST RECON
preds_dict = video_net(ref1, ref2, im, compress_type='train_compress', train_type='recon')
mse = mse_criterion(preds_dict['recon_image'], im)
mse.backward()
optimizer.step()

In [18]:
# TEST FULL
preds_dict = video_net(ref1, ref2, im, compress_type='train_compress', train_type='full')
mse = mse_criterion(preds_dict['recon_image'], im)
loss = mse * LAMBDA + preds_dict['bpp']
loss.backward()
optimizer.step()